In [20]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20320, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 20320 (delta 46), reused 52 (delta 23), pack-reused 20237 (from 1)
Receiving objects: 100% (20320/20320), 277.43 MiB | 35.08 MiB/s, done.
Resolving deltas: 100% (778/778), done.
Updating files: 100% (137/137), done.


In [4]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/

🔧 System Information
Python version: 3.12.12
PyTorch version: 2.9.0+cu126
CUDA version: 12.6
GPU count: 1
  GPU 0: Tesla T4
Wed Dec 31 16:28:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              

In [5]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch torchvision
# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown
!uv pip install wandb
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 26ms
Audited 1 package in 0.19ms
/content
Using Python 3.12.12 environment at: /usr
Audited 2 packages in 84ms

⬇️ Installing Dependencies (Manually fixed)...
  × No solution found when resolving dependencies:
  ╰─▶ Because torch==1.13.1 has no wheels with a matching Python ABI tag
      (e.g., `cp312`) and xformers==0.0.16 depends on torch==1.13.1, we can
      conclude that xformers==0.0.16 cannot be used.
      And because you require xformers==0.0.16, we can conclude that your
      requirements are unsatisfiable.

      hint: You require CPython 3.12 (`cp312`), but we only found wheels for
      `torch` (v1.13.1) with the following Python ABI tags: `cp37m`, `cp38`,
      `cp39`, `cp310`, `cp311`
Using Python 3.12.12 environment at: /usr
Resolved 47 packages in 89ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1

In [6]:
# KAGGLE CELL #1: Download checkpoint

import os
import sys
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

✅ Checkpoint already downloaded

📂 Files in ckpt/:
  ✓ unet.safetensors (300.34 MB)
  ✓ content_encoder.safetensors (4.54 MB)
  ✓ style_encoder.safetensors (78.58 MB)


In [7]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [8]:
import pandas as pd
import os
def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.
    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return
    all_characters = []
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")
convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)


--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /content/dummy_data.csv
Successfully converted '/content/dummy_data.csv' to '/content/dummy_chars.txt', with one character per line.


In [9]:
!ls -larth {OUTPUT_PATH}/ckpt
from huggingface_hub import login
HF_TOKEN = load_secret("HF_TOKEN")
login(HF_TOKEN)
HF_USERNAME = "dzungpham"

total 384M
drwxr-xr-x 3 root root 4.0K Dec 31 14:57 .cache
-rw-r--r-- 1 root root 4.6M Dec 31 14:57 content_encoder.safetensors
-rw-r--r-- 1 root root  79M Dec 31 14:57 style_encoder.safetensors
-rw-r--r-- 1 root root 301M Dec 31 14:57 unet.safetensors
drwxr-xr-x 3 root root 4.0K Dec 31 14:57 .
drwxr-xr-x 1 root root 4.0K Dec 31 16:28 ..
Attempting to load secret 'HF_TOKEN' from 'colab' environment...
✅ Successfully loaded secret 'HF_TOKEN'.


In [10]:
# %cd {OUTPUT_PATH}
# # ==========================================
# # EXPORT / DOWNLOAD DATASET COMMANDS
# # ==========================================

# # Train Split
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/train" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "train" \
#   --token HF_TOKEN
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/train_original" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "train_original" \
#   --token HF_TOKEN
# # Validation: Unseen Both
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/val_unseen_both" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "val_unseen_both" \
#   --token HF_TOKEN
# # Validation: Seen Style, Unseen Char
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/val_seen_style_unseen_char" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "val_seen_style_unseen_char" \
#   --token HF_TOKEN
# # Validation: Unseen Style, Seen Char
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/val_unseen_style_seen_char" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "val_unseen_style_seen_char" \
#   --token HF_TOKEN
# !python FontDiffusion/export_hf_dataset_to_disk.py \
#   --output_dir "my_dataset/test" \
#   --repo_id {HF_USERNAME}/font-diffusion-generated-data \
#   --split "test" \
#   --token HF_TOKEN
# print("SUCCESSFULLY EXPORT HF DATASET TO LOCAL DIRECTORY")

In [11]:
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 600 \
    --end_line 1000 \
    --batch_size 24 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 63ms
Prepared 2 packages in 0.28ms
Uninstalled 2 packages in 4ms
Installed 2 packages in 6ms
 - fsspec==2025.10.0
 + fsspec==2025.12.0
 - huggingface-hub==1.2.3
 + huggingface-hub==0.36.0
/content
/content/FontDiffusion/src/dpm_solver/dpm_solver_pytorch.py:53: SyntaxWarning: invalid escape sequence '\h'
  The `alphas_cumprod` is the \hat{alpha_n} arrays in the notations of DDPM. Specifically, DDPMs assume that
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER SYNTHESIS DATA GENERATION MAGIC
📖 Loading characters from file: FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt
   Lines 600 to 1000 (total file: 10174 lines)
   Processing 401 lines...
📖 Reading character file: 100%|███████████████████████████| 401/401 [00:00<00:00, 2016685.74line/s]
✅ Successfully loaded 401 single characters.

📂 Loading 15 style images from directory...
✓ Verifying style

In [22]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42


FONTDIFFUSION SPLIT CREATOR (SIMPLIFIED)
✓ Using source directory: my_dataset/train_original

CREATING DATA SPLITS

ANALYZING TRAINING DATA

🔍 Scanning target images...
Styles: 100%|██████████████████████████████████████████████████| 15/15 [00:00<00:00, 1208.97style/s]

✓ Initial scan:
  Styles: 15
  Characters: 141
  Valid (char, style) pairs: 2115

🔍 Auto-detecting font parameter from existing files...
  📂 Found 15 fonts in fonts/ directory
  ✓ Detected font parameter: 'HAN NOM A'
    Verified with: U+34E4_㓤_20242aa0.png

🔍 Validating content images (using font='HAN NOM A')...
Checking content: 100%|██████████████████████████████████████| 141/141 [00:00<00:00, 62114.99char/s]

⚠️  WARNING: 46/141 characters missing content images:
  Examples: ['𠄩', '𠑬', '𠤆', '𠦳', '𠳒', '𠴝', '𠺥', '𡊰', '𡗉', '𡥵']...
  These characters will be excluded from splits

✅ After validation:
  Characters with content images: 95
  Valid (char, style) pairs: 1425
  Content images found: 95

📊 Character distributi

In [ ]:
!uv pip install --upgrade pyarrow datasets

In [ ]:
# # --- RAW DATA (Before Splitting) ---
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/train_original" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "train_original" \
#   --private \
#   --token "{HF_TOKEN}"

# # --- ORGANIZED SPLITS (After Splitting) ---

# # Train Split
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/train" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "train" \
#   --private \
#   --token "{HF_TOKEN}"

# # Test Split
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/test" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "test" \
#   --private \
#   --token "{HF_TOKEN}"

# # Validation: Unseen Both
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/val_unseen_both" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "val_unseen_both" \
#   --private \
#   --token "{HF_TOKEN}"

# # Validation: Seen Style, Unseen Char
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/val_seen_style_unseen_char" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "val_seen_style_unseen_char" \
#   --private \
#   --token "{HF_TOKEN}"

# # Validation: Unseen Style, Seen Char
# !python FontDiffusion/create_hf_dataset.py \
#   --data_dir "my_dataset/val_unseen_style_seen_char" \
#   --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
#   --split "val_unseen_style_seen_char" \
#   --private \
#   --token "{HF_TOKEN}"
# print("SUCCESSFULLY UPLOAD LOCAL MY_DATASET TO HUGGINGFACE DATASETS SPACE")

In [23]:
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train_original" \
  --private \
  --token "{HF_TOKEN}"

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train" \
  --private \
  --token "{HF_TOKEN}"

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val" \
  --private \
  --token "{HF_TOKEN}"



FONTDIFFUSION DATASET CREATOR

Data dir: my_dataset/train_original
Repo: dzungpham/font-diffusion-generated-data
Push to Hub: True
✓ Validated directory structure
  Content images: my_dataset/train_original/ContentImage
  Target images: my_dataset/train_original/TargetImage
  Results checkpoint: my_dataset/train_original/results_checkpoint.json

BUILDING DATASET

✓ Loaded results_checkpoint.json
  Generations: 2175
  Characters: 401
  Styles: 15

🖼️  Loading 2175 image pairs...
Loading images: 100%|███████████████████████████████████████| 2175/2175 [00:01<00:00, 1242.16pair/s]
✓ Loaded 2175 samples

PUSHING TO HUB
Repository: dzungpham/font-diffusion-generated-data
Split: train_original
Uploading the dataset shards:   0% 0/1 [00:00<?, ? shards/s]
Map: 100% 2175/2175 [00:00<00:00, 28905.61 examples/s]

Creating parquet from Arrow format: 100% 1/1 [00:00<00:00, 26.14ba/s]
Processing Files (0 / 0)      : |          |  0.00B /  0.00B            
New Data Upload               : |          

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# TRAINING PHASE 1
!uv pip install "huggingface-hub==0.25.2"
import wandb

# Load your Weights & Biases API key from a secure store
wandb_key = load_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

# Run the training script with the corrected flag syntax
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name=FontDiffuser_training_phase_1 \
    --data_root=my_dataset \
    --output_dir=outputs/FontDiffuser \
    --report_to=wandb \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=200 \
    --ckpt_interval=100 \
    --val_interval=200 \
    --gradient_accumulation_steps=1 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler=linear \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision=no

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")